In [1]:
from config import setup
setup()

 # Projet AP GeoSafe, Evacuation lors de feu
 
 ## Generator 
 Pour générer les données utilisées après par le projet, il faut utiliser le générateur du projet evacsim

In [2]:
from docplex.cp.model import CpoModel
from docplex.cp.model import *
import numpy as np

configuration

In [3]:
from config_duc import setup
setup()

## modélisation

In [128]:
def geosafe_model(l, q, eps, r, W, H, d):
    '''
    Int  Matrix l: length path from i to j = G.l[i,j].
                        no path ij <--> G.l[i,j] = 0
    Int  Array  q: capacity of node i <--> G.q[i]
    Bool Array  eps: evacuation node 
    Bool Array  r: safe node
    
    Int  Array  W: initial population at node i <--> W[i]
    Int  H : Time span
    Int  Array  d: deadline to leave the node
    '''
    
    nb_node = q.shape[0]
    nodes = np.arange(nb_node) # id nodes, i.e [1,2,3,4, ...]
    total_population = np.sum(W)
    
    t = !eps & !r # transition node

    mdl = CpoModel(name='geosafe')
    
    # == Output ==
    # starting date
    s = np.array( mdl.integer_var_list(nb_node, min=0, max = 150, name="s") )
    
    # evacuation rate aka. height of package
    h = np.array( mdl.integer_var_list(nb_node, min=0, max = 300, name="h") )

    # == Intermediate ==
    # node flow of population
    phi = np.matrix([
            [ 
                mdl.integer_var(name='phi[%d,%d]'%(i,j)) for j in range(H) 
            ] for i in range(nb_node)
        ])
    
    # ending date (leaving time) of node
    e = np.array( mdl.integer_var_list(nb_node, min=0, max = 150, name= "e") )
    
    for v in range(nb_node):
        if eps[v]:
            mdl.add(
                e[v] == s[v] + W[v]/h[v]
            )
    
    # == Constraints ==
    # evacuate everyone in evacuation node
    for v in range(nb_node):
        mdl.add(
            mdl.sum(phi[v, t] for t in range (H)) == W[v] # TODO not really understand this constraint
        ) 
    
    # Flow at a node u = sum from all of its leaves node epsilon
#     mdl.add_constraint(
#         phi[u, t] == np.sum( phi[u, t - l[u,v] - s[eps]] ) \
#             for t in range(H)                                  \
#             for v in np.where(eps)                           \
#             for u in np.where(t)                             
#     )

    # simplified version
    for u in nodes[np.where(t)]:
        for t in range(H):
             
            v = nodes[np.where(
                                np.logical_and(
                                    l[u,:] > 0, 
                                    s + l[u,:] <= t,
                                    t <= e + l[u, :]
                                )
                               )[1]]             
            mdl.add(
                phi[u, t] == mdl.sum(h[v])
            )
    
    # Flow at a node does not excess capacity of arc
    for t in range(H):
        for u in nodes:
            mdl.add(
                phi[u,t] <= q[u]
            )
    
    # Objective
    mdl.add(
        minimize(np.max(s[eps] + h[eps]/W[eps] - d[eps]))
    )
    
    return mdl

In [35]:
a = np.array([True, True, False, False, True])
v = np.array([11,12,13,14,15])
print(v[np.where(a)])

[11 12 15]


## Unit test:

In [5]:
import networkx as nx

In [13]:
G = nx.Graph()

nodes_state = [
    (1, {'eps': True, 'r': False}),
    (2, {'eps': True, 'r': False}),
    (3, {'eps': True, 'r': False}),
    (4, {'eps': False, 'r': False}),
    (5, {'eps': False, 'r': False}),
    (6, {'eps': False, 'r': True})
]

edges_state = [
    (1, 4, {'l' : 4}),
    (2, 4, {'l' : 3}),
    (4, 5, {'l' : 7}),
    (3, 5, {'l' : 3}),
    (5, 6, {'l' : 10})
]

G.add_nodes_from(nodes_state)
G.add_edges_from(edges_state)

In [64]:
a = nx.get_node_attributes(G, 'eps').values()
for item in a:
    eps += item
print(eps)

[3. 3. 3. 3. 3. 3.]


- Init model

In [129]:
l = nx.to_numpy_matrix(G, weight='l')
q = np.array([20,20,20,20,20,20])
eps = np.array([True, True, True, False, False, False])
r = np.array([False, False, False, False, False, True])
W = np.array([5,5,5,0,0,0])
H = 150
d = np.array([150,150,150,150,150,150])

mdl = geosafe_model(l, q, eps, r, W, H, d)

- solve model

In [130]:
mdl.solve()
sol = mdl.solve()
if sol is not None:
    sol.print_solution()
else:
    print('Problem does not have solution')

CpoException: Can not execute command 'cpoptimizer -angel'. Please check availability of required executable file.